In [50]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.feature import StringIndexer
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.classification import NaiveBayes
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.classification import DecisionTreeClassifier


In [51]:
# Initialize Spark session
spark = SparkSession.builder.appName("irisClassification").getOrCreate()

In [52]:
iris_df=spark.read.csv("/content/Iris.csv",header=True,inferSchema=True)

In [53]:
iris_df.take(1)

[Row(Id=1, SepalLengthCm=5.1, SepalWidthCm=3.5, PetalLengthCm=1.4, PetalWidthCm=0.2, Species='Iris-setosa')]

In [54]:
VectorAssembler=VectorAssembler(inputCols=["SepalLengthCm","SepalWidthCm","PetalLengthCm","PetalWidthCm"],outputCol="features")

In [55]:
viris_df=VectorAssembler.transform(iris_df)

In [56]:
viris_df.take(1)

[Row(Id=1, SepalLengthCm=5.1, SepalWidthCm=3.5, PetalLengthCm=1.4, PetalWidthCm=0.2, Species='Iris-setosa', features=DenseVector([5.1, 3.5, 1.4, 0.2]))]

In [57]:
indexer=StringIndexer(inputCol="Species",outputCol="label")

In [58]:
iviris_df=indexer.fit(viris_df).transform(viris_df)

In [59]:
iviris_df.take(1)

[Row(Id=1, SepalLengthCm=5.1, SepalWidthCm=3.5, PetalLengthCm=1.4, PetalWidthCm=0.2, Species='Iris-setosa', features=DenseVector([5.1, 3.5, 1.4, 0.2]), label=0.0)]

In [60]:
splits=iviris_df.randomSplit([0.6,0.4],1)

In [61]:
train_df=splits[0]
test_df=splits[1]

In [62]:
train_df.count()

98

In [63]:
test_df.count()

52

In [64]:
iviris_df.count()

150

In [65]:
NaiveBayes=NaiveBayes(modelType="multinomial")

In [66]:
Nb_model=NaiveBayes.fit(train_df)

In [67]:
prediction_df=Nb_model.transform(test_df)

In [68]:
prediction_df.take(1)

[Row(Id=1, SepalLengthCm=5.1, SepalWidthCm=3.5, PetalLengthCm=1.4, PetalWidthCm=0.2, Species='Iris-setosa', features=DenseVector([5.1, 3.5, 1.4, 0.2]), label=0.0, rawPrediction=DenseVector([-12.111, -13.4138, -14.0841]), probability=DenseVector([0.7088, 0.1926, 0.0985]), prediction=0.0)]

In [69]:
evaluator=MulticlassClassificationEvaluator(labelCol="label",predictionCol="prediction",metricName="accuracy")

In [70]:
nbaccuracy=evaluator.evaluate(prediction_df)

In [71]:
nbaccuracy

0.9038461538461539

In [72]:
from pyspark.ml.classification import MultilayerPerceptronClassifier

In [73]:
layers=[4,5,5,3]

In [74]:
mlp=MultilayerPerceptronClassifier(layers=layers,seed=1)

In [75]:
mlp_evaluator=MulticlassClassificationEvaluator(metricName="accuracy")


In [76]:
mlp_accuracy=mlp_evaluator.evaluate(prediction_df)

In [77]:
mlp_accuracy

0.9038461538461539

In [78]:
dt=DecisionTreeClassifier(labelCol="label",featuresCol="features")

In [79]:
dt_model=dt.fit(train_df)

In [80]:
dt_prediction=dt_model.transform(test_df)

In [81]:
dt_evaluatopr=MulticlassClassificationEvaluator(labelCol="label",predictionCol="prediction",metricName="accuracy")

In [82]:
dt_accuracy=dt_evaluatopr.evaluate(dt_prediction)

In [83]:
dt_accuracy

0.9423076923076923